In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

In [3]:
def train_lightgbm_model(df, include_ratios=False, cv_folds=5):
    """
    Trains a LightGBM model on censored=0 data to predict final_expandEvts and validates on all data.
    Uses relative error metrics (sMAPE, sMdAPE, Normalized RMSE) and cross-validation.

    Parameters:
    - df (pandas.DataFrame): Preprocessed DataFrame with solver features, ratio features, and final_expandEvts.
    - include_ratios (bool): If True, includes ratio features; if False, excludes them (default: False).
    - cv_folds (int): Number of cross-validation folds (default: 5).

    Returns:
    - dict: Contains relative error metrics, CV scores, and feature importance for censored=0 test set and all data.
    """
    try:
        # Validate input
        if not isinstance(df, pd.DataFrame):
            raise TypeError(f"Input 'df' must be a pandas DataFrame, got {type(df)}")
        print(f"Input DataFrame type: {type(df)}")
        
        # Feature selection
        exclude_cols = ['filename', 'final_expandEvts', 'stop_iter', 'final_maxStackDepth']
        if not include_ratios:
            exclude_cols.extend([f'expandEvts_ratio_{i}' for i in range(1, 6)])
            exclude_cols.extend([f'pruneBacktrackEvts_ratio_{i}' for i in range(1, 6)])
        features = [col for col in df.columns if col not in exclude_cols]
        
        # Print target statistics
        print("\nTarget Statistics (final_expandEvts):")
        mean_censored = df[df['censored'] == 0]['final_expandEvts'].mean()
        std_censored = df[df['censored'] == 0]['final_expandEvts'].std()
        mean_all = df['final_expandEvts'].mean()
        std_all = df['final_expandEvts'].std()
        print(f"Mean (censored=0): {mean_censored:.2f}")
        print(f"Std Dev (censored=0): {std_censored:.2f}")
        print(f"Mean (all data): {mean_all:.2f}")
        print(f"Std Dev (all data): {std_all:.2f}")
        
        # Calculate relative error metrics
        def calculate_smape(y_true, y_pred):
            """Calculate Symmetric Mean Absolute Percentage Error (sMAPE)."""
            y_true, y_pred = np.array(y_true), np.array(y_pred)
            denominator = np.abs(y_true) + np.abs(y_pred)
            mask = denominator != 0  # Avoid division by zero
            if mask.sum() == 0:
                return np.nan
            smape = np.mean(2 * np.abs(y_true[mask] - y_pred[mask]) / denominator[mask]) * 100
            return smape
        
        def calculate_smdape(y_true, y_pred):
            """Calculate Symmetric Median Absolute Percentage Error (sMdAPE)."""
            y_true, y_pred = np.array(y_true), np.array(y_pred)
            denominator = np.abs(y_true) + np.abs(y_pred)
            mask = denominator != 0
            if mask.sum() == 0:
                return np.nan
            smape_terms = 2 * np.abs(y_true[mask] - y_pred[mask]) / denominator[mask] * 100
            return np.median(smape_terms)
        
        def calculate_normalized_rmse(y_true, y_pred, mean_y):
            rmse = np.sqrt(mean_squared_error(y_true, y_pred))
            return rmse / mean_y if mean_y != 0 else np.nan
        
        # Train on censored=0 data
        print("\nTraining LightGBM on censored=0 data...")
        df_censored = df[df['censored'] == 0]
        if df_censored.empty:
            print("Warning: No censored=0 instances found. Exiting.")
            return None
        
        X_censored = df_censored[features]
        y_censored = df_censored['final_expandEvts']
        X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X_censored, y_censored, test_size=0.2, random_state=42)
        
        lgb_model = lgb.LGBMRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
        lgb_model.fit(X_train_c, y_train_c)
        
        # Evaluate on censored=0 test set
        y_pred_c = lgb_model.predict(X_test_c)
        rmse_censored = np.sqrt(mean_squared_error(y_test_c, y_pred_c))
        norm_rmse_censored = calculate_normalized_rmse(y_test_c, y_pred_c, mean_censored)
        smape_censored = calculate_smape(y_test_c, y_pred_c)
        smdape_censored = calculate_smdape(y_test_c, y_pred_c)
        cv_scores = cross_val_score(lgb_model, X_censored, y_censored, cv=cv_folds, scoring='neg_root_mean_squared_error')
        cv_norm_rmse_censored = (-cv_scores.mean()) / mean_censored if mean_censored != 0 else np.nan
        print(f"Censored=0 Test Set RMSE: {rmse_censored:.4f}")
        print(f"Censored=0 Test Set Normalized RMSE: {norm_rmse_censored:.4%}")
        print(f"Censored=0 Test Set sMAPE: {smape_censored:.2f}%")
        print(f"Censored=0 Test Set sMdAPE: {smdape_censored:.2f}%")
        print(f"Censored=0 CV Normalized RMSE: {cv_norm_rmse_censored:.4%}")
        
        # Feature importance
        importance = pd.DataFrame({
            'feature': X_censored.columns,
            'importance': lgb_model.feature_importances_
        }).sort_values('importance', ascending=False)
        print("\nCensored=0 Feature Importance (Top 10):")
        print(importance.head(10))
        
        # Validate on all data
        print("\nValidating on all data...")
        X_all = df[features]
        y_all = df['final_expandEvts']
        y_pred_all = lgb_model.predict(X_all)
        rmse_all = np.sqrt(mean_squared_error(y_all, y_pred_all))
        norm_rmse_all = calculate_normalized_rmse(y_all, y_pred_all, mean_all)
        smape_all = calculate_smape(y_all, y_pred_all)
        smdape_all = calculate_smdape(y_all, y_pred_all)
        print(f"All Data RMSE: {rmse_all:.4f}")
        print(f"All Data Normalized RMSE: {norm_rmse_all:.4%}")
        print(f"All Data sMAPE: {smape_all:.2f}%")
        print(f"All Data sMdAPE: {smdape_all:.2f}%")

        # Cross-validation for all data
        print("\nPerforming cross-validation on all data...")
        cv_scores_all = cross_val_score(lgb_model, X_all, y_all, cv=cv_folds, scoring='neg_root_mean_squared_error')
        cv_norm_rmse_all = (-cv_scores_all.mean()) / mean_all if mean_all != 0 else np.nan
        print(f"All Data CV Normalized RMSE: {cv_norm_rmse_all:.4%}")
        
        return {
            'rmse_censored_test': rmse_censored,
            'norm_rmse_censored_test': norm_rmse_censored,
            'smape_censored_test': smape_censored,
            'smdape_censored_test': smdape_censored,
            'cv_norm_rmse_censored': cv_norm_rmse_censored,
            'rmse_all': rmse_all,
            'norm_rmse_all': norm_rmse_all,
            'smape_all': smape_all,
            'smdape_all': smdape_all,
            'cv_norm_rmse_all': cv_norm_rmse_all,
            'feature_importance': importance,
        }
    
    except Exception as e:
        print(f"Error in train_lightgbm_model: {e}")
        return None

df = pd.read_excel("structured_data.xlsx")
results = train_lightgbm_model(df, include_ratios=True)
results

Input DataFrame type: <class 'pandas.core.frame.DataFrame'>

Target Statistics (final_expandEvts):
Mean (censored=0): 53227800.02
Std Dev (censored=0): 174281183.67
Mean (all data): 761155886.29
Std Dev (all data): 636531467.58

Training LightGBM on censored=0 data...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000778 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2975
[LightGBM] [Info] Number of data points in the train set: 239, number of used features: 62
[LightGBM] [Info] Start training from score 44120478.410042
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning]

{'rmse_censored_test': 149992189.49292123,
 'norm_rmse_censored_test': 2.8179295298124223,
 'smape_censored_test': 165.4726965435505,
 'smdape_censored_test': 199.99897528049362,
 'cv_norm_rmse_censored': 1.8985397143264189,
 'rmse_all': 628498959.9308732,
 'norm_rmse_all': 0.825716481011541,
 'smape_all': 121.14329190880844,
 'smdape_all': 98.77469448632903,
 'cv_norm_rmse_all': 0.11914730305654105,
 'feature_importance':                        feature  importance
 48           coef_of_variation         149
 44                     min_num          66
 47          range_to_avg_ratio          62
 40                   total_sum          51
 42                    skewness          50
 ..                         ...         ...
 50  pruneBacktrackEvts_ratio_1           0
 49          expandEvts_ratio_1           0
 1                       evts_1           0
 36            strengthenEvts_5           0
 70   diff_pruneBacktrackEvts_5           0
 
 [71 rows x 2 columns]}